# First Geopandas Example

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd


# Loading shapefiles for the Italian regions

In [ ]:
data_directory= "../Data/"

regioni_gdf = gpd.read_file(data_directory+"Istat confini amministrativi 2023/Reg01012023")
regioni_gdf

showing the regions' shapes

In [ ]:
regioni_gdf.plot(edgecolor='black', linewidth=1.5, figsize=(10,8))

In [ ]:
# overlay of regions

In [ ]:
print(regioni_gdf.columns.values)

In [ ]:
province_gdf = gpd.read_file(data_directory+"Istat confini amministrativi 2023/ProvCM01012023")
province_gdf.plot(edgecolor='black', linewidth=1.1, figsize=(10,8))


In [ ]:
# save memory space 
province_gdf = None

# Reading shapefile into a GeoDataFrame
comuni_gdf = gpd.read_file(data_directory+"Istat confini amministrativi 2023/Com01012023")
comuni_gdf.plot(edgecolor='red', linewidth=0.8, figsize=(10,8))


In [ ]:
# save space in memory
comuni_gdf = None

regioni_gdf.plot(figsize=(10,15), edgecolor='black', linewidth=1.5, column='DEN_REG', legend=True, cmap='Set3')

# Displaying different features on the map

In [ ]:
region_capita_df = pd.read_csv(data_directory+'Region data/regions.income.data.csv')
region_population_df = pd.read_csv(data_directory+'Region data/regions.population.data.csv', encoding='latin-1')
region_income_population = pd.merge(region_population_df, region_capita_df, left_on='regione', right_on='Region', how='inner')
region_income_population.drop(columns=['Region'], inplace=True)

# Merge the income dataframe with the region geoframe
region_total_data_gdf = regioni_gdf.merge(region_income_population, left_on='DEN_REG', right_on='regione', how='left')

In [ ]:
region_population_df

# Visualise the income on the region shapefile 

In [ ]:
# Plotting the GeoDataFrame with colors based on 'Income'
fig, ax = plt.subplots(1, 1, figsize=(10, 8))  # Adjust the figure size as needed
region_total_data_gdf.plot(column='Income', ax=ax, legend=True, cmap='YlGn', edgecolor='black')
plt.title('Income by Region')  # Set the plot title
plt.show()

## Visualise the population on the region map 

In [ ]:
# Plotting the GeoDataFrame with colors based on 'residenti'
fig, ax = plt.subplots(1, 1, figsize=(10, 8))  # Adjust the figure size as needed
region_total_data_gdf.plot(column='residenti', ax=ax, legend=True, cmap='YlGn', edgecolor='black')
plt.title('Residents by Region')  # Set the plot title


# Visualising the density of population

In [ ]:
# Plotting the GeoDataFrame with colors based on 'densità'
fig, ax = plt.subplots(1, 1, figsize=(10, 8))  # Adjust the figure size as needed
region_total_data_gdf.plot(column='densita', ax=ax, legend=True, cmap='gist_earth', edgecolor='black')
plt.title('Density by Region')  # Set the plot title


# Defining a region of interest

In [ ]:
dati_epistat_gdf = gpd.read_file(data_directory+"DATI_EPISAT/DATI_EPISAT.shp")
dati_epistat_gdf

In [ ]:
print(dati_epistat_gdf.columns.values)

In [ ]:
# the data about the pianura padana have ZONACLIM==2.0
pianura_padana_gdf=dati_epistat_gdf.loc[dati_epistat_gdf["ZONACLIM"]==2.0]
pianura_padana_gdf.plot(figsize=(10,15), edgecolor='black', linewidth=1.5)
pianura_padana_gdf

In [ ]:
# Overlap the shape of Pianura Padana wiht the Italian regions

In [ ]:
# the Coordinate Reference Systems (CRS) of both geodataframes must match. So you map them (just to be sure)
pianura_padana_gdf = pianura_padana_gdf.to_crs(regioni_gdf.crs)

# Plot the map
fig, ax = plt.subplots(figsize=(10, 10))
regioni_gdf.plot(ax=ax, color="lightgrey", edgecolor="black", alpha=0.8)  # Base map
pianura_padana_gdf.plot(ax=ax, color="red", alpha=0.5)  # Overlay Pianura Padana

# Add title and legend
plt.title("Pianura Padana Overlaid on Italy Map")
plt.show()


In [ ]:
# free up memory or PyCharm will get very slow
comuni_gdf = None
province_gdf = None
dati_epistat_gdf = None

# Overlaying the two maps  
but this time by colouring the regions   

In [ ]:
# Create a Matplotlib figure and axis
fig, ax = plt.subplots(figsize=(10, 8))  # Adjust figsize as needed

# Plot 'regioni_gdf' GeoDataFrame
regioni_gdf.plot(ax=ax, color='lightgrey', edgecolor='black', linewidth=0.5, column='DEN_REG', legend=True, cmap='Set3')
# Plot 'new_row' GeoDataFrame
pianura_padana_gdf.plot(ax=ax, color='orange', edgecolor='black', linewidth=1.5)
plt.show()

## Make pianura padana transparent
So to visualise the underlying regions using alpha close to zero

In [ ]:
# Create a Matplotlib figure and axis
fig, ax = plt.subplots(figsize=(10, 16))  # Adjust figsize as needed

# Plot 'regioni_gdf' GeoDataFrame
regioni_gdf.plot(ax=ax,  edgecolor='black', linewidth=0.9, column='DEN_REG', legend=True, cmap='Set3')
# Plot 'new_row' GeoDataFrame
pianura_padana_gdf.plot(ax=ax, edgecolor='yellow', linewidth=1.5, alpha=0.1)
plt.show()

# Creating an intersection region 
We will use the overlay function which - given two geopandas dataframes, returns a combined geopandas dataframe.
The behaviour depends on the parameter "how"
how: Defines the type of overlay operation. Common options include:

- intersection: Returns geometries where df1 and df2 overlap.
- union: Combines geometries from both GeoDataFrames, retaining all areas.
- difference: Retains areas of df1 that do not overlap with df2.
- symmetric_difference: Combines geometries that are in df1 or df2 but not in both. 

In [ ]:
intersected = gpd.overlay(pianura_padana_gdf, regioni_gdf, how='intersection')
intersected

In [ ]:
# plot the regions in the intersection area - we use a different cmap - the previous one would not show good colours in this example
intersected.plot(column='DEN_REG', legend=True, cmap='tab20b', figsize=(10, 8))  


# Showing the map with Folium and adding the regions
we will use GeoJson
The folium.GeoJson function is used to add GeoJSON data (geometric shapes, points, lines, or polygons) to a Folium map. It transforms spatial data from a GeoPandas GeoDataFrame into a format Folium can render on an interactive map.
Example: folium.GeoJson(regioni_gdf, name='Regioni').add_to(m)
(note the add_to_map part)


In [ ]:
import folium

# Assuming 'regioni_gdf' and 'intersected' are your GeoPandas DataFrames

# Create a Folium Map centered at a specific location (e.g., [latitude, longitude])
# we use Milan as center
Milan = [45.4642, 9.1900]
m = folium.Map(location=Milan, zoom_start=6)  # Adjust the location and zoom level as needed

# Add 'regioni_gdf' GeoPandas DataFrame to the map
folium.GeoJson(regioni_gdf, name='Regioni').add_to(m)

# Add 'intersected' GeoPandas DataFrame to the map
folium.GeoJson(intersected, name='Pianura Padana', ).add_to(m)

# allow adding.removing layers
folium.LayerControl().add_to(m)
# Display the map
m

# Displaying the roads in a specific city

In [ ]:
file_path = data_directory+'ne_10m_roads/ne_10m_roads.shp'

city_roads = gpd.read_file(file_path)
city_roads= city_roads.loc[city_roads['continent']=='Europe']
city_roads

In [ ]:
import folium

# Milan again
city_center= [45.4642, 9.1900]
# Create a Folium Map centered at the city's location
m = folium.Map(location=city_center, zoom_start=9)

# Add city roads to the Folium map
folium.GeoJson(city_roads).add_to(m)

# Display the map
m

# Choropleth map example    

In [ ]:
Rome= [41.9028, 12.4964]
# Create a Folium Map centered at coordinates [latitude, longitude]
m = folium.Map(location=Rome, zoom_start=5)  

# Add Choropleth layer
folium.Choropleth(
    geo_data=region_total_data_gdf,  #  your GeoDataFrame
    name='Choropleth',
    data=region_total_data_gdf,  #  your GeoDataFrame
    columns=['regione', 'Income'],  # Specify the region column and income column
    key_on='feature.properties.DEN_REG',  # Key in GeoJSON properties
    fill_color='YlGn',  # Choose a color scale
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Income'  # Legend label
).add_to(m)

# Display the map
m



# A map of regions in Spain

In [ ]:
regioni_gdf = gpd.read_file("../Data/geoJson/es.json")
regioni_gdf.plot(edgecolor='black', linewidth=1.5, figsize=(10,8))

# geojson overlay example

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Load the improved GeoJSON file into a GeoDataFrame
geojson_path = "../Data/geoJson/geojson example.json"
overlay_gdf = gpd.read_file(geojson_path)

# Ensure the GeoJSON has the same CRS as the regions shapefile
overlay_gdf = overlay_gdf.to_crs(regioni_gdf.crs)
# Plot the base layer (regions)
fig, ax = plt.subplots(figsize=(10, 8))
regioni_gdf.plot(ax=ax, edgecolor='black', linewidth=1.5, color='lightgrey', alpha=0.7)

# Overlay the GeoJSON layer with custom styling
overlay_gdf.plot(
    ax=ax,
    column="type",  # Use the 'type' property for color differentiation
    cmap="Set3",
    edgecolor="red",
    linewidth=1.2,
    alpha=0.9,  # Transparency for the overlay
    legend=True
)

# Add labels for the GeoJSON features
for x, y, label in zip(overlay_gdf.geometry.centroid.x, overlay_gdf.geometry.centroid.y, overlay_gdf["name"]):
    ax.text(x, y, label, fontsize=9, ha="center", bbox=dict(facecolor="red", alpha=0.5, edgecolor="red"))

# Add title and grid
plt.title("Overlay of GeoJSON Features on Italian Regions", fontsize=14)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True)

plt.show()